In [42]:
import tensorflow as tf
import numpy as np
import pandas as pd


# Step-1: Create tf dataset from a list

In [43]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31,12,26,18,] # -108 is an error because daily sales number can not be negative 
daily_sales_numbers

[21, 22, -108, 31, -1, 32, 34, 31, 12, 26, 18]

In [44]:
data=tf.data.Dataset.from_tensors(daily_sales_numbers) # Or
data

<TensorDataset element_spec=TensorSpec(shape=(11,), dtype=tf.int32, name=None)>

In [45]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers) # create a simple tf dataset from a python list
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [46]:
for sales in tf_dataset:
    print(sales)# individual elements here is in a tensor


tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(26, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)


In [47]:
for sales in tf_dataset:   # or
    print(sales.numpy())   # here converting  this tensor into a numpy

21
22
-108
31
-1
32
34
31
12
26
18


In [48]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)  

21
22
-108
31
-1
32
34
31
12
26
18


    | Iterate through first n elements in tf dataset

In [49]:
for sales in tf_dataset.take(3): # return only specific element
    print(sales.numpy())             


21
22
-108


    | Filter sales numbers that are < 0

In [50]:
tf_dataset=tf_dataset.filter(lambda x: x>0)

In [51]:
for sales in tf_dataset.as_numpy_iterator(): # fitering negative data
    print(sales)


21
22
31
32
34
31
12
26
18


    | Convert sales numbers from USA dollars ($) to Bangladeshi Taka (BDT) Assuming 1->80 conversation rate


In [52]:
tf_dataset = tf_dataset.map(lambda x: x*80) # converting doller to taka

In [53]:
for sales in tf_dataset.as_numpy_iterator(): 
    print(sales)

1680
1760
2480
2560
2720
2480
960
2080
1440


    | Shuffling

In [54]:
tf_datasett=tf_dataset.shuffle(3) # shuffling elements
for sales in tf_dataset.as_numpy_iterator(): 
    print(sales)

1680
1760
2480
2560
2720
2480
960
2080
1440


    | Batching

In [94]:
for sales_batch in tf_dataset.batch(4): # creating batch according to specific batch of size
    print(sales_batch.numpy())

[[1584 2232]
 [1512 2448]
 [2232  864]
 [2304 1872]]
[[1296]]


# Step-2: use of tensorflow input pipeline

    | Perform all of the above operations in one shot

In [56]:

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers) 
tf_dataset=tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)

In [57]:
for sales in tf_dataset.as_numpy_iterator(): 
    print(sales)

[1512 2232]
[2304 2448]
[2232 1584]
[1872 1296]
[864]


    | Read images using tensorflow input pipeline

In [58]:
# Read images
images_dataset=tf.data.Dataset.list_files(r'images/*/*', shuffle=False)
images_dataset


<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [59]:
for file in images_dataset.take(6): # here it store image path 
    print(file.numpy())

b'images\\Animal\\0c701217ce0d151a4048f559b3613070.jpg'
b'images\\Animal\\383dd362b1dde98b5f28cb71b54238bd.jpg'
b'images\\Animal\\7f2bcfec799d4bc0e57e5fd42af3518c--white-lions-white-tigers.jpg'
b'images\\Animal\\8c97042a7caf01e2c7609f33b5c17e55.jpg'
b'images\\Animal\\HD-wallpaper-dreamy.jpg'
b'images\\Animal\\cute-baby-chicks-animal-wp1iuazbb451hz2c.jpg'


In [60]:
images_dataset=images_dataset.shuffle(200) # shuffling
for file in images_dataset.take(5):
    print(file.numpy())


b'images\\Gold fish\\a-Ryukin-Goldfish-in-aquarium_dien_Shutterstock.jpg'
b'images\\Gold fish\\feeder-goldfish_JuanCarlosPalauDiaz_Shutterstock.png'
b'images\\Animal\\images (15).jpeg'
b'images\\Gold fish\\final_coral_reef_03 (1).webp'
b'images\\Gold fish\\1.jpg'


In [61]:
images_count=len(images_dataset)
images_count

100

In [62]:
class_name=['Animal','Gold fish']

# Step-3: Train test split

    | Note:
    Here x_train is actual image data and ytrain is image label(Animal, gold fish)

In [63]:
train_size=int(images_count*0.8)
train_size

80

In [64]:
train_data=images_dataset.take(train_size)
train_data

test_data=images_dataset.skip(train_size) # oposit of take
test_data

<SkipDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [65]:
len(train_data)

80

In [66]:
len(test_data)

20

    | Retrive Label (Fish & Animal)

In [67]:
s='images\\Animal\\0c701217ce0d151a4048f559b3613070.jpg'
s.split('\\')

['images', 'Animal', '0c701217ce0d151a4048f559b3613070.jpg']

In [68]:
s.split('\\')[-2]

'Animal'

In [69]:
for t in train_data.take(4):
    print(t.numpy())

b'images\\Gold fish\\download (1).jpeg'
b'images\\Gold fish\\7.jpg'
b'images\\Animal\\images (15).jpeg'
b'images\\Gold fish\\98032132f77f209890e1120374e9f281--funny-iphone-wallpaper-desktop-wallpapers.jpg'


    | Retrive y_train | label (Animal,Fish)

In [70]:
import os

In [71]:
def get_label(file_path):
    path= tf.strings.split(file_path,os.path.sep) # os.path.sep means os seperator which seperator the file path
    return path[-2] 

In [72]:
get_label('images\\Animal\\0c701217ce0d151a4048f559b3613070.jpg')

<tf.Tensor: shape=(), dtype=string, numpy=b'Animal'>

In [73]:
for label in train_data.map(get_label):
    print(label.numpy())

b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Animal'
b'Animal'
b'Animal'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Animal'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Animal'
b'Animal'
b'Animal'
b'Animal'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Animal'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Animal'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Animal'
b'Gold fish'
b'Gold fish'
b'Animal'
b'Gold fish'
b'Animal'
b'Animal'


In [74]:
def process_image(file_path):
    label=get_label(file_path)
    img=tf.io.read_file(file_path) # load the raw data from the file as a string
    img=tf.image.decode_jpeg(img)
    img=tf.image.resize(img,[128,128]) # resize the image
    return img, label

In [75]:
img, label = process_image('images\\Animal\\0c701217ce0d151a4048f559b3613070.jpg')
img.numpy()[:2]

array([[[254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [254.     , 254.     , 254.     ],
        [25

    | Retrive x_train

In [76]:
train_data = train_data.map(process_image)
test_data = test_data.map(process_image)

In [77]:
for image, label in train_data.take(1):
    print("**** Image",image)
    print("**** Label",label)

**** Image tf.Tensor(
[[[154.66016  186.23438  214.23438 ]
  [151.02344  185.21484  212.40625 ]
  [147.81366  183.2633   211.51953 ]
  ...
  [212.28516  208.23828  205.20038 ]
  [214.7099   210.7757   209.84149 ]
  [212.30133  210.30133  211.30133 ]]

 [[144.98047  180.70312  209.27734 ]
  [141.57812  178.875    207.44922 ]
  [138.40625  177.68317  207.3711  ]
  ...
  [214.24219  210.1684   208.34375 ]
  [210.33514  207.62994  208.12213 ]
  [208.82874  207.27734  209.20148 ]]

 [[137.0055   179.       210.8711  ]
  [134.34375  177.17188  209.04297 ]
  [132.08405  176.86731  209.83215 ]
  ...
  [215.04889  210.13269  213.78528 ]
  [204.00739  202.17188  207.01477 ]
  [202.80743  201.79803  207.65234 ]]

 ...

 [[196.78046  178.78046   94.78046 ]
  [223.81793  205.81793  119.81793 ]
  [211.2995   194.2995   104.2995  ]
  ...
  [129.81201  110.81201   33.807983]
  [203.74841  184.74841  107.74841 ]
  [141.12976  122.12976   45.17273 ]]

 [[165.32776  147.32776   63.6604  ]
  [156.62543  1

    | Scaling

In [78]:
def scale(image, label):
    return image/255, label

In [79]:
train_data = train_data.map(scale)


In [80]:
for image, label in train_data.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.98039216 0.98039216 0.98039216]
****Label:  b'Gold fish'
****Image:  [0.14901961 0.13725491 0.11764706]
****Label:  b'Gold fish'
****Image:  [0.02184436 0.06106005 0.01792279]
****Label:  b'Gold fish'
****Image:  [0.13498774 0.14283088 0.09185049]
****Label:  b'Animal'
****Image:  [0.37573528 0.43063724 0.29338235]
****Label:  b'Gold fish'
